In [1]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import math
import torchtext
from torch.nn.functional import softmax as softmax
from torch.nn.functional import log_softmax as logsoftmax
from os.path import exists
import sys
import random

In [2]:
import itertools

In [3]:
torch.__version__

'1.11.0'

In [4]:
GPU_NUM = 1# 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU

print ('# Current cuda device: ', torch.cuda.current_device())

# Current cuda device:  1


In [5]:
def GPUreport():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"# DEVICE {i}: {torch.cuda.get_device_name(i)}")
            print("- Memory Usage:")
            print(f"  Allocated: {round(torch.cuda.memory_allocated(i)/1024**3,1)} GB")
            print(f"  Cached:    {round(torch.cuda.memory_cached(i)/1024**3,1)} GB\n")
    else:
        print("# GPU is not available")
GPUreport()

# DEVICE 0: NVIDIA GeForce RTX 3090
- Memory Usage:
  Allocated: 0.0 GB
  Cached:    0.0 GB

# DEVICE 1: NVIDIA GeForce RTX 3090
- Memory Usage:
  Allocated: 0.0 GB
  Cached:    0.0 GB



/opt/conda/envs/Conda1/lib/python3.9/site-packages/torch/cuda/memory.py:384: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [6]:
device = (torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu'))
#device = torch.device('cpu')#CPu로 할떄

torch.set_printoptions(sci_mode=True)

In [7]:
torchtext.__version__

'0.12.0'

In [8]:
#!conda install datasets #기록용. 주석처리 풀지말것
#!pip install torch
#!pip install ipywidgets
#!pip install spacy
#!pip install torchtext
#!python -m spacy download en_core_web_sm


In [9]:
#데이터셋 불러오기
from datasets import load_dataset
wmt16 = load_dataset('wmt16', 'de-en')
wmt16['test'][33]

Reusing dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0)


  0%|          | 0/3 [00:00<?, ?it/s]

{'translation': {'de': 'Sein Auto lief noch in der Einfahrt.',
  'en': 'His car was still running in the driveway.'}}

데이터셋 테스트

In [10]:
t1= wmt16['test'][:5]
for ding in t1['translation']:
    print(ding)

{'de': 'Obama empfängt Netanyahu', 'en': 'Obama receives Netanyahu'}
{'de': 'Das Verhältnis zwischen Obama und Netanyahu ist nicht gerade freundschaftlich.', 'en': 'The relationship between Obama and Netanyahu is not exactly friendly.'}
{'de': 'Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.', 'en': "The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East."}
{'de': 'Bei der Begegnung soll es aber auch um den Konflikt mit den Palästinensern und die diskutierte Zwei-Staaten-Lösung gehen.', 'en': 'The meeting was also planned to cover the conflict with the Palestinians and the disputed two state solution.'}
{'de': 'Das Verhältnis zwischen Obama und Netanyahu ist seit Jahren gespannt.', 'en': 'Relations between Obama and Netanyahu have been strained for years.'}


In [11]:
wmt16['validation'][:]['translation'][20:30]

[{'de': 'Rückbesinnung auf das sehr amerikanische Ideal der Arbeitsrechte als Menschenrechte',
  'en': 'Reconnecting With the Very American Ideal That Labor Rights Are Human Rights'},
 {'de': 'Die Kongressabgeordneten Keith Ellison und John Lewis haben einen Gesetzesvorschlag eingebracht, um die Organisation von Gewerkschaften als Bürgerrecht zu etablieren.',
  'en': 'Congressmen Keith Ellison and John Lewis have proposed legislation to protect union organizing as a civil right.'},
 {'de': '"So wie Gewerkschaften sterben, sterben auch die Mittelklassejobs," sagte Ellison, ein Demokrat aus Minnesota und stellvertretender Vorsitzender des Progressive Caucus im Kongress.',
  'en': '"As go unions, so go middle-class jobs," says Ellison, the Minnesota Democrat who serves as a Congressional Progressive Caucus co-chair.'},
 {'de': 'Daher stelle ich stolz gemeinsam mit der Bürgerrechtsikone John Lewis das Mitarbeiterermächtigungsgesetz vor.',
  'en': "That's why I'm proud to introduce the Empl

In [12]:
len(wmt16['train']), wmt16['validation'], wmt16['test'],

(4548885,
 Dataset({
     features: ['translation'],
     num_rows: 2169
 }),
 Dataset({
     features: ['translation'],
     num_rows: 2999
 }))

In [13]:
#문장 토큰화
#데이터셋이 너무 커서, 코드짜는 도중에는 1퍼센트만 사용하겠다.
"""
DATA_PERCENTAGE = 0.005
import spacy
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

def build_vocab(dataset, tokenizer, languege, percentage):
    wordCounter = Counter()
    with tqdm(total = math.trunc(len(dataset['train'])*percentage/100)
              + math.trunc(len(dataset['validation'])*percentage/100)
              + math.trunc(len('test')*percentage/100)) as progressBar:
        for setNames in ['validation', 'test', 'train']:
            setSize = math.trunc(len(dataset[setNames])*percentage/100)
            totalsetSize = len(dataset[setNames])
            print(setNames, "set", setSize, totalsetSize, setSize*100/totalsetSize)
            proCounter = 0
            if setSize > 0: 
                for translation in dataset[setNames]:#Each 문장 pair
                    wordCounter.update([token.text for token in tokenizer(translation['translation'][languege])])
                    progressBar.update(1)
                    proCounter += 1
                    if proCounter==setSize:
                        print(setNames, proCounter, "next set")
                        break
    sorted_by_freq_tuples = sorted(wordCounter.items(), key=lambda x:x[1], reverse=True)
    ordered_Dict = OrderedDict(sorted_by_freq_tuples)
    return vocab(ordered_Dict, specials= ['<sos>', "<eos>","<pad>"])
en_vocab = build_vocab(wmt16, spacy_en, 'en', DATA_PERCENTAGE)
de_vocab = build_vocab(wmt16, spacy_de, 'de', DATA_PERCENTAGE)
torch.save(en_vocab, "en_vocab dp = "+str(DATA_PERCENTAGE)+"percent.pt")
torch.save(de_vocab, "de_vocab dp = "+str(DATA_PERCENTAGE)+"percent.pt")
"""

'\nDATA_PERCENTAGE = 0.005\nimport spacy\nfrom collections import Counter, OrderedDict\nfrom torchtext.vocab import vocab\n\nspacy_en = spacy.load(\'en_core_web_sm\')\nspacy_de = spacy.load(\'de_core_news_sm\')\n\ndef build_vocab(dataset, tokenizer, languege, percentage):\n    wordCounter = Counter()\n    with tqdm(total = math.trunc(len(dataset[\'train\'])*percentage/100)\n              + math.trunc(len(dataset[\'validation\'])*percentage/100)\n              + math.trunc(len(\'test\')*percentage/100)) as progressBar:\n        for setNames in [\'validation\', \'test\', \'train\']:\n            setSize = math.trunc(len(dataset[setNames])*percentage/100)\n            totalsetSize = len(dataset[setNames])\n            print(setNames, "set", setSize, totalsetSize, setSize*100/totalsetSize)\n            proCounter = 0\n            if setSize > 0: \n                for translation in dataset[setNames]:#Each 문장 pair\n                    wordCounter.update([token.text for token in tokenizer(tr

In [14]:
import spacy
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')
en_vocab = torch.load("./en_vocab dp = 10percent.pt")
de_vocab = torch.load("./de_vocab dp = 10percent.pt")

In [15]:
temp = []
for sentence_pair in t1['translation']:
    ttemp = [token.text for token in spacy_en(sentence_pair['en'])]
    temp.append(ttemp)

In [16]:
temp#Not wordpiece

[['Obama', 'receives', 'Netanyahu'],
 ['The',
  'relationship',
  'between',
  'Obama',
  'and',
  'Netanyahu',
  'is',
  'not',
  'exactly',
  'friendly',
  '.'],
 ['The',
  'two',
  'wanted',
  'to',
  'talk',
  'about',
  'the',
  'implementation',
  'of',
  'the',
  'international',
  'agreement',
  'and',
  'about',
  'Teheran',
  "'s",
  'destabilising',
  'activities',
  'in',
  'the',
  'Middle',
  'East',
  '.'],
 ['The',
  'meeting',
  'was',
  'also',
  'planned',
  'to',
  'cover',
  'the',
  'conflict',
  'with',
  'the',
  'Palestinians',
  'and',
  'the',
  'disputed',
  'two',
  'state',
  'solution',
  '.'],
 ['Relations',
  'between',
  'Obama',
  'and',
  'Netanyahu',
  'have',
  'been',
  'strained',
  'for',
  'years',
  '.']]

In [17]:
#실제 데이터 생성. 데이터: 사전인 en_vocab 과 de_vocab
#pad
FILENAME_DATA = "data.pt"

SOS_IDX = en_vocab["<sos>"]
EOS_IDX = en_vocab["<eos>"]
PAD_IDX_EN = en_vocab["<pad>"]
PAD_IDX_DE = de_vocab["<pad>"]

def data_process(dataset):
    tokenized_data = []
    for sentence_pair in dataset['translation']:
        de_tensor_ = torch.tensor(de_vocab([token.text for token in spacy_de(sentence_pair['de'])]), dtype=torch.long)
        en_tensor_ = torch.tensor(en_vocab(['<sos>',]+[token.text for token in spacy_en(sentence_pair['en'])]+['<eos>']), dtype=torch.long)
        tokenized_data.append((de_tensor_, en_tensor_))
    return tokenized_data


if exists(FILENAME_DATA):
    print(FILENAME_DATA, "exists")
    sample_data = torch.load("./"+FILENAME_DATA)
else:
    sample_data = data_process(wmt16['train'][:math.trunc(len(wmt16['train'])*9.99/100)])
    #sample_data = data_process(wmt16['train'][:10000])
    torch.save(sample_data, "data.pt")

data.pt exists


In [18]:
def gpusize(tensor):
    return tensor.element_size() * tensor.nelement()/1e+6

In [19]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
def generate_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        en_batch.append(en_item)
        de_batch.append(de_item)
    de_batch = pad_sequence(de_batch, padding_value=PAD_IDX_DE).transpose(0,1)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX_EN).transpose(0,1)
    return de_batch, en_batch

In [20]:
sample_dataloader = DataLoader(sample_data, batch_size=10, shuffle=True, collate_fn=generate_batch)
sample_batch=None
for batch in sample_dataloader:
    
    sample_batch=batch
    break
sample_batch[0]#독일어

tensor([[  910,     3,   151,     5,   416,    12,    20,  2486, 14229,  1952,
           149,     3,   407,    24,    23,    12,   215,     3,    47,   356,
         47814,     4,     2,     2,     2,     2,     2,     2,     2,     2,
             2],
        [ 5223,  2826,    45,   279,     3,    20, 20996,  8292,  2242,     3,
           627,   636,     6,  2560,   102, 16628,   284,   412, 55457,  6836,
             4,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2],
        [   34,   157,  2580,     3,    75,    24,     5,   557, 16629,     3,
            63,    10,   984,     6,   527,     3,    87, 11084,     3,   175,
         21000,  1251,     9,   191,     4,     2,     2,     2,     2,     2,
             2],
        [   39,    58,     3,    24,   616,   103,    73,     3,  3571,   661,
            59,    98,  5264,    13,     4,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
 

In [21]:
sample_batch[1]

tensor([[    0,    66,  1106,   189,     7,     3,  3449,   219,     8,     7,
             3, 12403,     8, 22656,  6088,     5,  2101,     3,   407,    15,
             3,   556,   392,  2683,    22,  3089,     4,    13,   153,    24,
            11,    92,     7,   783,     3,   139,  1962,  2023,     5,     1,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2],
        [    0,   443,    10,    88,     4,   712,    55,   810,    77,     3,
          1299,    14,    12,  7785,  1739,     6,  2130,     5,    70,    75,
           400,     4,    12,  1121,    36,    97,  5828,    11,     7,  1768,
           258,    54,    20,     3,  1121,    11,  3466,     5,     1,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2],
        [    0,   103,  2887,    85,    32,    43,     4,    13,    98,   727,
            13,    22,    48,   237,     3,   380,     7,   100,    12,   303,
    

In [22]:
#HYPERPARAMETERS
embed_size = 50
hidden_size_ex = 25

In [23]:
de_word_size = len(de_vocab)
en_word_size = len(en_vocab)
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(de_word_size, embed_size).to(device = device)#word_sizexembed size matrix. 정수->벡터 함수
        self.LSTM = torch.nn.LSTM(input_size = embed_size , hidden_size= hidden_size_ex, batch_first=True).to(device = device)
 
    def forward(self, x):
        x = x.to(device = device)#x shape: batch*sentence_size
        #print("Encoder input:", x)#debug
        x = self.embed(x).to(device = device)#x.shape = batchxsentence size(문장의 토큰 개수)x embed_size
        #print(token.unsqueeze(0).unsqueeze(0))#1x1x4
        output, final_status = self.LSTM(x)
        #print("Encoder output:", final_status)#debug
        return output, final_status# shape: batch x sentence_size(LSTM이 돈 횟수) x hidden_size_sx

In [24]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(en_word_size, embed_size).to(device = device)#word_sizexembed size matrix. 정수->벡터 함수
        self.LSTM = torch.nn.LSTM(input_size = embed_size , hidden_size= hidden_size_ex, batch_first=True).to(device = device)
    def forward(self, x, final_encoder_state):
        x = x.to(device = device)#x shape: batch*sentence_size
        #print("Decoder input:", x)#debug
        x = self.embed(x)#x.shape = batchxsentence size(문장의 토큰 개수)x embed_size
        output, final_decoder_status = self.LSTM(x, final_encoder_state)
        #print("Decoder output:", final_decoder_status)#debug
        return output, final_decoder_status# shape: batch x sentence_size(LSTM이 돈 횟수) x hidden_size_sx

In [25]:
class Attention_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder=Decoder()
        self.encoder=Encoder()
        self.hiddenstate = nn.Sequential(nn.Linear(hidden_size_ex*2, hidden_size_ex), nn.BatchNorm1d(hidden_size_ex)).to(device = device)
        self.outputLayer = nn.Linear(hidden_size_ex, en_word_size).to(device = device)
        
    def forward(self, x):#x=(de,en) sentence imbedding
        tf = random.randrange(2)
        ground_truth = x[1].to(device = device)
        #print("ALSTM en ground_truth:", ground_truth)#debug
        en_sentence_size = x[1].shape[1]
        #print("ALSTM en ground_truth sentence size:", en_sentence_size)#debug
        en_generated = torch.zeros((ground_truth.shape[0],1),requires_grad = False, dtype=torch.int64).to(device=device)
        output= None
        de_batch = x[0].to(device = device)
        #print("ALSTM de input:", de_batch)#debug
        encoder_result, final_encoder_status = self.encoder(de_batch)#batch*sentence size*embedding
        final_status = final_encoder_status #state for decoder...     
        for iteration in range(1, en_sentence_size):
            decoder_result, final_status = self.decoder(en_generated[:,-1].unsqueeze(1), final_status)#decoder result: batch*1*hidden_status
            attention_score = torch.matmul(encoder_result,decoder_result.transpose(1,2))#decoder_result를 column vector로 바꿔주었다. final shape = batch*encoder sentence size* 1(디코더에는 한 번에 한 token만 넣어주므로..)
            attention_distribution = softmax(attention_score, dim=1)
            #print("ALSTM attention_distribution:", attention_distribution)#debug
            attention_value = torch.sum(encoder_result * attention_distribution, dim=1).unsqueeze(1)#아다마르 곱 후 unsqueeze 로 batch*1*hidden_size 로 바꿔준다
            #print("ALSTM attention_value:", attention_value)#debug
            att_dec_concat = torch.cat((attention_value, decoder_result), dim = 2).squeeze(1)#batch*hidden_sie*2
            #print("ALSTM input for final Linear layer:", att_dec_concat)#debug
            final_vector = torch.tanh(self.hiddenstate(att_dec_concat))#batch*hidden_s
            a_output = self.outputLayer(final_vector)#batch*en_word_size
            #print("ALSTM output of final Linear layer:", output)#debug
            #output_softmax = softmax(output, dim=1)
            #output_lsm = logsoftmax(output,dim=1)
            #print("ALSTM Logsoftmax:", output_lsm)#debug
            
            a_output_argmax = a_output.argmax(dim=1).view(-1,1)
            current_truth = ground_truth[:,iteration]
            
            #print(iteration, output_softmax[:,current_truth].shape)
            #print(current_truth)

            if self.training == True:#what to do in training mode -> model.train()...
                if  tf == 1:
                    en_generated = torch.cat((en_generated, current_truth.unsqueeze(1)), dim=1)#teacher forcing
                else:
                    en_generated = torch.cat((en_generated, a_output_argmax), dim=1)#Non_teacher_forcing
            else:
                en_generated = torch.cat((en_generated, a_output_argmax), dim=1)
            #prob_truth = output_lsm[[idx for idx in range(output_lsm.shape[0])],current_truth].unsqueeze(1)
            if output is None:#shape = batch*sentence_size2
                output = a_output.unsqueeze(1)#batch*1*en 사전크기
                #print(output.shape)
            else:
                output = torch.cat((output, a_output.unsqueeze(1)),dim=1)
                #print(output.shape)
            #print(en_generated)
        #print("tensor size of output: ", gpusize(output))# debug
        return en_generated, output
sample_LSTM = Attention_LSTM()
sample_LSTM.eval()
sample_generated, sample_output = sample_LSTM(sample_batch)
sample_LSTM.train()
sample_LSTM = None

In [26]:
sample_generated.reshape(-1).shape

torch.Size([380])

In [27]:
sample_output.reshape(-1,en_word_size).shape

torch.Size([370, 59564])

In [26]:
def training_loop(optimizer, model, loss_fn, train_loader):
    model.train()
    loss_train = 0.0
    for batch in train_loader:
        generated_sentence, logit = model(batch)
        ground_truth = batch[1].to(device = device)
        ground_truth = torch.split(ground_truth, [1,ground_truth.shape[1]-1], dim=1)[1]#train 이라서 ground truth. 제일 처음의 0을 제거함.
        ground_truth = ground_truth.reshape(-1)
        logit = logit.reshape(-1,en_word_size)
        mask = ground_truth != 2# 2인경우에는 0
        #print(generated_sentence.dtype)
        #print(prob_output)#debug
        #logsoftmax_stack = logsoftmax_stack.to(device=device)
        #loss = loss_fn(logsoftmax_stack.reshape(-1,logsoftmax_stack.shape[-1]), generated_sentence.reshape(-1))
        loss = loss_fn(logit, ground_truth)*mask
        #print(loss, mask)
        loss = loss.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss
        #print("loss:",loss)
        #break
    loss_train = loss_train/len(train_loader)
    torch.save(model.state_dict(), FILENAME_PARAMETERS)
    #print("batchloss",loss_train)
    return loss_train

In [27]:
FILENAME_PARAMETERS = "seq2seq_param.pt"
LR = 0.004
EPOCH = 0
print(device)
#train----------------------------------
model = Attention_LSTM()
if exists(FILENAME_PARAMETERS):
    model.load_state_dict(torch.load("./" + FILENAME_PARAMETERS))
    print("param exists")
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay = 0.004)
loss_fn = nn.CrossEntropyLoss(reduce = False, reduction = 'none')
#train this fold
train_loader = DataLoader(sample_data, batch_size=80, shuffle=True, collate_fn=generate_batch)
for epoch in range(EPOCH):
    result = training_loop(optimizer, model, loss_fn, train_loader)
    print("epoch", epoch, " loss:", result)

cuda:1
param exists


/opt/conda/envs/Conda1/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [ ]:
GPUreport()

In [ ]:
torch.save(model.state_dict(), FILENAME_PARAMETERS)

In [28]:
model.eval()

Attention_LSTM(
  (decoder): Decoder(
    (embed): Embedding(59564, 50)
    (LSTM): LSTM(50, 25, batch_first=True)
  )
  (encoder): Encoder(
    (embed): Embedding(175559, 50)
    (LSTM): LSTM(50, 25, batch_first=True)
  )
  (hiddenstate): Sequential(
    (0): Linear(in_features=50, out_features=25, bias=True)
    (1): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (outputLayer): Linear(in_features=25, out_features=59564, bias=True)
)

In [29]:
sample_eval = model(sample_batch)[0].to(device = 'cpu')

In [30]:
torch.cuda.empty_cache()

In [31]:
dic = en_vocab.get_itos()

In [32]:
def batch_to_string(tensor):
    tensorlist = tensor.tolist()
    strings = []
    for string in tensorlist:
        temp1 = ''
        for letter in string:
            if letter:
                temp1 += ' ' + dic[letter]
        strings.append(temp1)
    return strings

In [33]:
batch_to_string(sample_eval)

[' Obnova is to the negotiations negotiations in the negotiations negotiations in the negotiations negotiations in the negotiations negotiations in the negotiations in the negotiations in the negotiations and the negotiations negotiations in the negotiations and the negotiations negotiations in the negotiations and the negotiations negotiations in the negotiations and the negotiations',
 ' publication are to the reversed of the the of the the of the of the of the of the of the of the of the the of the of the of the of the of the of the of the of the of the of the of the of the of',
 ' rapporteurs agree with Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs Mrs',
 ' President Mr President , I - that , that is , , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that , that',


In [34]:
batch_to_string(sample_batch[1])

[' This applies both to the interim agreement and to the Hebron and Wye Agreements . While the negotiations on the final peace settlement have begun , I think it is important to keep the two processes apart . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
 " At that time , she was talking about the desire for a reversed burden of proof . In other words , a product ' s manufacturer is to demonstrate whether or not the product is hazardous . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>",
 " ( NL ) Mr President , I am pleased I have been given the opportunity to make a few introductory remarks at the end of rapporteur Mrs Berger ' s speech , after which I will , of course , listen with all due attention to the next speakers . <eos>",
 ' Mr President , this report is positive but , actually , we need more than this . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

In [ ]:
emb = next(model.decoder.embed.parameters())

In [ ]:
torch.set_printoptions(sci_mode=False)
emb[17]

In [ ]:
dic[17]

In [ ]:
sample_eval